# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
EPS = 1e-15

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("ACC： ",accuracy_score(true_edges, predict_edges))
    #print("AP： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("RECALL： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=LR, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [4]:
def get_graph(nodes_number):
    edges = pd.read_csv('datasets/email.csv', header=None).iloc[1:,]
    edges = edges.astype('int')
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[0]):
        graph_np[edges.iloc[i, 0], edges.iloc[i, 1]]=1
        graph_np[edges.iloc[i, 1], edges.iloc[i, 0]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.7),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.7)*6,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [5]:
nodes_number = 986

In [6]:
embedding_dim = 12
n_clusters = 24
alpha = 1
beta = 4.5
DEC = 5.0
LR = 0.1

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask = get_graph(nodes_number)
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(300):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.7487296217882328
AUC SCORE:  0.7397213159208692
AP：  0.7643415360079017
AUC SCORE:  0.7551523518878807
######################### Module cycle ： 1 ##########################
AP：  0.7718201018739719
AUC SCORE:  0.7554534185017476
AP：  0.7618603558281332
AUC SCORE:  0.7404616828222654
######################### Module cycle ： 2 ##########################
AP：  0.7623290659343271
AUC SCORE:  0.7392103700005165
AP：  0.7795512600563759
AUC SCORE:  0.7613236256262805
######################### Module cycle ： 3 ##########################
AP：  0.7941595738211243
AUC SCORE:  0.7798894862347412
AP：  0.8020374489399896
AUC SCORE:  0.7905644703087069
######################### Module cycle ： 4 ##########################
AP：  0.8117066003543991
AUC SCORE:  0.8023050567311169
AP：  0.815300235572736
AUC SCORE:  0.8068990719856752
######################### Module cycle ： 5 ##########################
AP：  0.820608918893276
AUC SCOR

AP：  0.9068015843548348
AUC SCORE:  0.899826234052444
######################### Module cycle ： 46 ##########################
AP：  0.9071217232800948
AUC SCORE:  0.9001701503073294
AP：  0.9071802090538262
AUC SCORE:  0.900233983574663
######################### Module cycle ： 47 ##########################
AP：  0.9075015599662868
AUC SCORE:  0.9005693342056782
AP：  0.9075427115053827
AUC SCORE:  0.9006125927583891
######################### Module cycle ： 48 ##########################
AP：  0.9078325809329812
AUC SCORE:  0.900930575403316
AP：  0.9079071864566731
AUC SCORE:  0.9010033832062121
######################### Module cycle ： 49 ##########################
AP：  0.9082020858015766
AUC SCORE:  0.9012969172362735
AP：  0.9082670756605706
AUC SCORE:  0.9013449751209518
######################### Module cycle ： 50 ##########################
AP：  0.908558647962523
AUC SCORE:  0.9016426843546081
AP：  0.9086308405570485
AUC SCORE:  0.9017074860970025
######################### Module cycle ： 51 

AP：  0.9171366838221062
AUC SCORE:  0.9126760902877018
AP：  0.9171995249464463
AUC SCORE:  0.9127855494912277
######################### Module cycle ： 92 ##########################
AP：  0.9172818079982691
AUC SCORE:  0.9128928995712883
AP：  0.9173439766715046
AUC SCORE:  0.9129959453177459
######################### Module cycle ： 93 ##########################
AP：  0.9174325802282158
AUC SCORE:  0.913106695821353
AP：  0.917481789313993
AUC SCORE:  0.9131920507567018
######################### Module cycle ： 94 ##########################
AP：  0.9175717147685621
AUC SCORE:  0.9133050179921144
AP：  0.917595503114335
AUC SCORE:  0.9133731340713831
######################### Module cycle ： 95 ##########################
AP：  0.9177221507278193
AUC SCORE:  0.9135190509805272
AP：  0.9177051216432701
AUC SCORE:  0.9135262822609804
######################### Module cycle ： 96 ##########################
AP：  0.9179129662792488
AUC SCORE:  0.9137704671062826
AP：  0.9178900815722888
AUC SCORE:  0.91378

AP：  0.9214156628113681
AUC SCORE:  0.9189508186842514
######################### Module cycle ： 137 ##########################
AP：  0.9214850115264849
AUC SCORE:  0.9190688435116475
AP：  0.921520281880247
AUC SCORE:  0.919129922005475
######################### Module cycle ： 138 ##########################
AP：  0.921569007126139
AUC SCORE:  0.9192026222000311
AP：  0.9215943604684333
AUC SCORE:  0.9192339147053252
######################### Module cycle ： 139 ##########################
AP：  0.9216198845364053
AUC SCORE:  0.9192811332449509
AP：  0.921659966793902
AUC SCORE:  0.9193176770372411
######################### Module cycle ： 140 ##########################
AP：  0.9216787773058508
AUC SCORE:  0.9193509495359931
AP：  0.9217206823363359
AUC SCORE:  0.9193910013601693
######################### Module cycle ： 141 ##########################
AP：  0.921714851035518
AUC SCORE:  0.919414137153286
AP：  0.9217421196195885
AUC SCORE:  0.9194515848556326
######################### Module cycle ： 

AP：  0.9227588907879847
AUC SCORE:  0.9212127675143335
AP：  0.9227771748563439
AUC SCORE:  0.9212456311013928
######################### Module cycle ： 183 ##########################
AP：  0.9227778705824715
AUC SCORE:  0.9212466641414576
AP：  0.9227966473180771
AUC SCORE:  0.9212775262133917
######################### Module cycle ： 184 ##########################
AP：  0.9227951473308267
AUC SCORE:  0.9212745562232054
AP：  0.922818716979174
AUC SCORE:  0.921307398288597
######################### Module cycle ： 185 ##########################
AP：  0.9228241821501325
AUC SCORE:  0.9213069248119006
AP：  0.9228453416924791
AUC SCORE:  0.9213395731822799
######################### Module cycle ： 186 ##########################
AP：  0.9228483716277689
AUC SCORE:  0.9213398960073003
AP：  0.9228636674153378
AUC SCORE:  0.9213688426507808
######################### Module cycle ： 187 ##########################
AP：  0.9228666868351407
AUC SCORE:  0.9213686919991046
AP：  0.9228864070526532
AUC SCORE:  0

...

AP：  0.9242437615425991
AUC SCORE:  0.9176586577366092
AP：  0.9242747429263168
AUC SCORE:  0.9177376422582256
######################### Module cycle ： 82 ##########################
AP：  0.9242847546383965
AUC SCORE:  0.9177707210619652
AP：  0.9243244109032331
AUC SCORE:  0.9178554088256056
######################### Module cycle ： 83 ##########################
AP：  0.9243236310301128
AUC SCORE:  0.917885216335807
AP：  0.9243601352052013
AUC SCORE:  0.917977824073277
######################### Module cycle ： 84 ##########################
AP：  0.9243620344589439
AUC SCORE:  0.9180058022416968
AP：  0.9243960906922697
AUC SCORE:  0.9181012938826811
######################### Module cycle ： 85 ##########################
AP：  0.9243854045013813
AUC SCORE:  0.9181291859644289
AP：  0.9244209940545345
AUC SCORE:  0.9182334799676314
######################### Module cycle ： 86 ##########################
AP：  0.9244244027990385
AUC SCORE:  0.918273488748472
AP：  0.924463403778186
AUC SCORE:  0.918385

AP：  0.9257250087873066
AUC SCORE:  0.9216256391935401
######################### Module cycle ： 127 ##########################
AP：  0.9257141233759318
AUC SCORE:  0.9216416728362116
AP：  0.92571413810094
AUC SCORE:  0.921664614934316
######################### Module cycle ： 128 ##########################
AP：  0.9257131486588169
AUC SCORE:  0.9216881381174566
AP：  0.9257152518620244
AUC SCORE:  0.9217092078304437
######################### Module cycle ： 129 ##########################
AP：  0.9257137882848893
AUC SCORE:  0.9217339147053253
AP：  0.9257217080380543
AUC SCORE:  0.9217587076668792
######################### Module cycle ： 130 ##########################
AP：  0.9257241793247618
AUC SCORE:  0.9217807888982628
AP：  0.9257233750124826
AUC SCORE:  0.9218005027461648
######################### Module cycle ： 131 ##########################
AP：  0.9257214271039191
AUC SCORE:  0.9218240904943097
AP：  0.9257231003591886
AUC SCORE:  0.9218471186790861
######################### Module cycle

AP：  0.9260780891326605
AUC SCORE:  0.9233865635922246
AP：  0.9260830001787148
AUC SCORE:  0.9234061913534548
######################### Module cycle ： 173 ##########################
AP：  0.9260860859511959
AUC SCORE:  0.9234174256641587
AP：  0.9260903047994913
AUC SCORE:  0.9234353316919475
######################### Module cycle ： 174 ##########################
AP：  0.9260937457024123
AUC SCORE:  0.9234476851293882
AP：  0.9260991634699454
AUC SCORE:  0.9234660861555414
######################### Module cycle ： 175 ##########################
AP：  0.9261030210667328
AUC SCORE:  0.9234783750279781
AP：  0.9261138783801671
AUC SCORE:  0.9235002410426818
######################### Module cycle ： 176 ##########################
AP：  0.9261160820536691
AUC SCORE:  0.9235105714433292
AP：  0.9261218201433313
AUC SCORE:  0.9235283483411098
######################### Module cycle ： 177 ##########################
AP：  0.9261241539945043
AUC SCORE:  0.9235413474285912
AP：  0.9261262511744968
AUC SCORE: 

AP：  0.9262695114655327
AUC SCORE:  0.9245981904581533
######################### Module cycle ： 218 ##########################
AP：  0.926267746236406
AUC SCORE:  0.9246059597803068
AP：  0.92626880705119
AUC SCORE:  0.9246187651727759
######################### Module cycle ： 219 ##########################
AP：  0.9262729236646383
AUC SCORE:  0.9246281916633667
AP：  0.9262737835949175
AUC SCORE:  0.9246409324908318
######################### Module cycle ： 220 ##########################
AP：  0.9262759931621442
AUC SCORE:  0.9246514781081593
AP：  0.9262757862041326
AUC SCORE:  0.9246628200272035
######################### Module cycle ： 221 ##########################
AP：  0.9262791241749064
AUC SCORE:  0.9246734732528711
AP：  0.926277483029928
AUC SCORE:  0.9246849443019232
######################### Module cycle ： 222 ##########################
AP：  0.9262791655181863
AUC SCORE:  0.924694564487526
AP：  0.9262816586822167
AUC SCORE:  0.924705648146554
######################### Module cycle ： 

AP：  0.9264535153030153
AUC SCORE:  0.9256130662350854
AP：  0.926449577643861
AUC SCORE:  0.9256244727191335
######################### Module cycle ： 264 ##########################
AP：  0.9264549507583217
AUC SCORE:  0.9256319192162669
AP：  0.9264586987304069
AUC SCORE:  0.9256469628622097
######################### Module cycle ： 265 ##########################
AP：  0.926463090187885
AUC SCORE:  0.9256588858662902
AP：  0.9264676484649246
AUC SCORE:  0.9256700556119901
######################### Module cycle ： 266 ##########################
AP：  0.9264706738951474
AUC SCORE:  0.9256786642791962
AP：  0.9264718249058148
AUC SCORE:  0.9256907164132849
######################### Module cycle ： 267 ##########################
AP：  0.9264696083924696
AUC SCORE:  0.925697904650402
AP：  0.9264773297263703
AUC SCORE:  0.9257139598147414
######################### Module cycle ： 268 ##########################
AP：  0.9264765615815251
AUC SCORE:  0.9257217937018991
AP：  0.926481154743438
AUC SCORE:  0.9

In [8]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.9249387195630238
AP FIRST AP STD:  0.004619898693959707
AP FIRST AUC MEAN:  0.9234707714226685
AP FIRST AUC STD:  0.0039948098500647185
AUC FIRST AP MEAN:  0.9248473751821568
AUC FIRST AP STD:  0.004543072825046282
AUC FIRST AUC MEAN:  0.923869600213495
AUC FIRST AUC STD:  0.004106336286805026
